In [ ]:
pip install apscheduler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
import requests
from transformers import pipeline
from flask import Flask, request, jsonify
from apscheduler.schedulers.background import BackgroundScheduler
import sqlite3

In [ ]:
import requests
from transformers import pipeline
import sqlite3

# Database setup
def init_db():
    conn = sqlite3.connect('news.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS news (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT,
                    summary TEXT,
                    sentiment TEXT,
                    url TEXT
                 )''')
    conn.commit()
    conn.close()

# Fetch NewsAPI
API_KEY = '23a12afbb4494529836f3675cf9176da'
NEWS_API_URL = 'https://newsapi.org/v2/top-headlines?country=us&apiKey=' + API_KEY

# Initialize NLP pipelines
summarizer = pipeline('summarization', model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline('sentiment-analysis')

# Fetch and process news
def fetch_and_process_news():
    print("Fetching and processing news...")
    response = requests.get(NEWS_API_URL)
    if response.status_code == 200:
        news_data = response.json().get('articles', [])

        conn = sqlite3.connect('news.db')
        c = conn.cursor()

        for article in news_data:
            title = article.get('title', '')
            content = article.get('content', '')
            url = article.get('url', '')

            if content:
                # Summarize content
                summary = summarizer(content, max_length=50, min_length=25, do_sample=False)[0]['summary_text']

                # Analyze sentiment
                sentiment = sentiment_analyzer(summary)[0]['label']

                # Store in database
                c.execute("INSERT INTO news (title, summary, sentiment, url) VALUES (?, ?, ?, ?)",
                          (title, summary, sentiment, url))

        conn.commit()
        conn.close()
        print("News data fetched and stored in the database.")
    else:
        print("Failed to fetch news. Check your API key or internet connection.")

# Display summarized news with sentiment
def display_news(sentiment_filter=None):
    conn = sqlite3.connect('news.db')
    c = conn.cursor()

    if sentiment_filter:
        c.execute("SELECT title, summary, sentiment, url FROM news WHERE sentiment = ?", (sentiment_filter,))
    else:
        c.execute("SELECT title, summary, sentiment, url FROM news")

    rows = c.fetchall()
    conn.close()

    if rows:
        for row in rows:
            print(f"Title: {row[0]}")
            print(f"Summary: {row[1]}")
            print(f"Sentiment: {row[2]}")
            print(f"URL: {row[3]}\n{'-'*50}\n")
    else:
        print("No news available.")

# Main function
def main():
    init_db()

    while True:
        print("\nNews Summarizer")
        print("1. Fetch and process news")
        print("2. Display all news")
        print("3. Display positive news")
        print("4. Display negative news")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            fetch_and_process_news()
        elif choice == "2":
            display_news()
        elif choice == "3":
            display_news(sentiment_filter="POSITIVE")
        elif choice == "4":
            display_news(sentiment_filter="NEGATIVE")
        elif choice == "5":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



News Summarizer
1. Fetch and process news
2. Display all news
3. Display positive news
4. Display negative news
5. Exit
Enter your choice: 1


Your max_length is set to 50, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


Fetching and processing news...


Your max_length is set to 50, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


News data fetched and stored in the database.

News Summarizer
1. Fetch and process news
2. Display all news
3. Display positive news
4. Display negative news
5. Exit
Enter your choice: 2
Title: Supreme Court dismisses Nvidia’s appeal in securities lawsuit - The Hill
Summary: The Supreme Court on Wednesday dismissed an appeal from Nvidia in a securities lawsuit against the chipmaking giant, allowing the case to move forward.
Sentiment: POSITIVE
URL: https://thehill.com/regulation/court-battles/5034515-supreme-court-nvidia-case-dismissed/
--------------------------------------------------

Title: Alexander brothers arrested in Miami - The Real Deal
Summary: A lawyer representing Oren and Alon confirmed the three brothers were arrested in Miami Wednesday. A video obtained by The Real Deal showed FBI agents out.
Sentiment: NEGATIVE
URL: https://therealdeal.com/national/2024/12/11/tal-oren-and-alon-alexander-arrested/
--------------------------------------------------

Title: NFL odds, lin

In [ ]:
import requests
from transformers import pipeline
import sqlite3
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np

# Function to initialize and load models only once (using a checkpoint method)
def load_models():
    global summarizer, sentiment_analyzer, category_classifier
    summarizer = pipeline('summarization', model="facebook/bart-large-cnn")
    sentiment_analyzer = pipeline('sentiment-analysis')

    # Initialize simple categorization model using Naive Bayes
    category_classifier = pipeline('zero-shot-classification')
    print("Models loaded successfully.")

# Database setup function (this only runs once to initialize the DB)
def init_db():
    conn = sqlite3.connect('news.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS news (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT,
                    summary TEXT,
                    sentiment TEXT,
                    category TEXT,
                    url TEXT
                 )''')
    conn.commit()
    conn.close()

# Fetch NewsAPI data using the provided API key
def fetch_and_process_news(API_KEY):
    NEWS_API_URL = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={23a12afbb4494529836f3675cf9176da}'
    print("Fetching and processing news...")
    response = requests.get(NEWS_API_URL)
    if response.status_code == 200:
        news_data = response.json().get('articles', [])

        conn = sqlite3.connect('news.db')
        c = conn.cursor()

        for article in news_data:
            title = article.get('title', '')
            content = article.get('content', '')
            url = article.get('url', '')

            if content:
                # Summarize content
                summary = summarizer(content, max_length=50, min_length=25, do_sample=False)[0]['summary_text']

                # Analyze sentiment
                sentiment = sentiment_analyzer(summary)[0]['label']

                # Categorize news
                category = categorize_news(title + ' ' + summary)  # Combine title and summary for categorization

                # Store in database
                c.execute("INSERT INTO news (title, summary, sentiment, category, url) VALUES (?, ?, ?, ?, ?)",
                          (title, summary, sentiment, category, url))

        conn.commit()
        conn.close()
        print("News data fetched and stored in the database.")
    else:
        print("Failed to fetch news. Check your API key or internet connection.")

# Simple categorization using pre-defined categories
def categorize_news(text):
    categories = ['Politics', 'Technology', 'Sports', 'Business', 'Health', 'Entertainment']
    result = category_classifier(text, candidate_labels=categories)
    return result['labels'][0]  # Return the most likely category

# Display summarized news with sentiment filter options
def display_news(sentiment_filter=None, category_filter=None):
    conn = sqlite3.connect('news.db')
    c = conn.cursor()

    query = "SELECT title, summary, sentiment, category, url FROM news"
    params = []

    if sentiment_filter:
        query += " WHERE sentiment = ?"
        params.append(sentiment_filter)
    if category_filter:
        query += " AND category = ?" if sentiment_filter else " WHERE category = ?"
        params.append(category_filter)

    c.execute(query, tuple(params))
    rows = c.fetchall()
    conn.close()

    if rows:
        for row in rows:
            print(f"Title: {row[0]}")
            print(f"Summary: {row[1]}")
            print(f"Sentiment: {row[2]}")
            print(f"Category: {row[3]}")
            print(f"URL: {row[4]}\n{'-'*50}\n")
    else:
        print("No news available.")

# Main function where you enter your API Key and get the desired results
def main():
    # Load models once
    load_models()

    # Initialize the database once
    init_db()

    # API Key (you can also load this from a config file or environment variable)
    API_KEY = input("Enter your NewsAPI key: ").strip()

    while True:
        print("\nNews Summarizer")
        print("1. Fetch and process news")
        print("2. Display all news")
        print("3. Display positive news")
        print("4. Display negative news")
        print("5. Display news by category")
        print("6. Display news by sentiment and category")
        print("7. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            fetch_and_process_news(API_KEY)
        elif choice == "2":
            display_news()
        elif choice == "3":
            display_news(sentiment_filter="POSITIVE")
        elif choice == "4":
            display_news(sentiment_filter="NEGATIVE")
        elif choice == "5":
            category_filter = input("Enter category (Politics, Technology, Sports, etc.): ").strip()
            display_news(category_filter=category_filter)
        elif choice == "6":
            sentiment_filter = input("Enter sentiment (POSITIVE, NEGATIVE): ").strip()
            category_filter = input("Enter category (Politics, Technology, Sports, etc.): ").strip()
            display_news(sentiment_filter=sentiment_filter, category_filter=category_filter)
        elif choice == "7":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


In [ ]:
import requests
from transformers import pipeline
import sqlite3
import os

# Function to initialize and load models only once (using a checkpoint method)
def load_models():
    # This function will load models once and return them.
    global summarizer, sentiment_analyzer
    summarizer = pipeline('summarization', model="facebook/bart-large-cnn")
    sentiment_analyzer = pipeline('sentiment-analysis')
    print("Models loaded successfully.")

# Database setup function (this only runs once to initialize the DB)
def init_db():
    conn = sqlite3.connect('news.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS news (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT,
                    summary TEXT,
                    sentiment TEXT,
                    url TEXT
                 )''')
    conn.commit()
    conn.close()

# Fetch NewsAPI data using the provided API key
def fetch_and_process_news(API_KEY):
    NEWS_API_URL = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={23a12afbb4494529836f3675cf9176da}'
    print("Fetching and processing news...")
    response = requests.get(NEWS_API_URL)
    if response.status_code == 200:
        news_data = response.json().get('articles', [])

        conn = sqlite3.connect('news.db')
        c = conn.cursor()

        for article in news_data:
            title = article.get('title', '')
            content = article.get('content', '')
            url = article.get('url', '')

            if content:
                # Summarize content
                summary = summarizer(content, max_length=50, min_length=25, do_sample=False)[0]['summary_text']

                # Analyze sentiment
                sentiment = sentiment_analyzer(summary)[0]['label']

                # Store in database
                c.execute("INSERT INTO news (title, summary, sentiment, url) VALUES (?, ?, ?, ?)",
                          (title, summary, sentiment, url))

        conn.commit()
        conn.close()
        print("News data fetched and stored in the database.")
    else:
        print("Failed to fetch news. Check your API key or internet connection.")

# Display summarized news with sentiment filter options
def display_news(sentiment_filter=None):
    conn = sqlite3.connect('news.db')
    c = conn.cursor()

    if sentiment_filter:
        c.execute("SELECT title, summary, sentiment, url FROM news WHERE sentiment = ?", (sentiment_filter,))
    else:
        c.execute("SELECT title, summary, sentiment, url FROM news")

    rows = c.fetchall()
    conn.close()

    if rows:
        for row in rows:
            print(f"Title: {row[0]}")
            print(f"Summary: {row[1]}")
            print(f"Sentiment: {row[2]}")
            print(f"URL: {row[3]}\n{'-'*50}\n")
    else:
        print("No news available.")

# Main function where you enter your API Key and get the desired results
def main():
    # Load models once
    load_models()

    # Initialize the database once
    init_db()

    # API Key (you can also load this from a config file or environment variable)
    API_KEY = input("Enter your NewsAPI key: ").strip()

    while True:
        print("\nNews Summarizer")
        print("1. Fetch and process news")
        print("2. Display all news")
        print("3. Display positive news")
        print("4. Display negative news")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            fetch_and_process_news(API_KEY)
        elif choice == "2":
            display_news()
        elif choice == "3":
            display_news(sentiment_filter="POSITIVE")
        elif choice == "4":
            display_news(sentiment_filter="NEGATIVE")
        elif choice == "5":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()
